<a href="https://colab.research.google.com/github/DovTech/models_roofs_experiments/blob/main/yolov8_format_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
import albumentations as A

import os
import gdown

In [ ]:
!pip install ultralytics --quiet
from ultralytics import YOLO
from ultralytics.data.converter import convert_coco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.7/616.7 kB 8.2 MB/s eta 0:00:00


# Files load

In [ ]:
gdown.download("https://drive.google.com/file/d/1KrbT2KbayuNtsACy7GHmGbUWtikpbSfI/view?usp=sharing", fuzzy=True) #train_small (img + msk)
gdown.download("https://drive.google.com/file/d/1MpYdtlbfdCoKW1K1qTcqA_en6RnBtb6g/view?usp=drive_link", fuzzy=True) #val_small (img + msk)
#gdown.download("https://drive.google.com/file/d/1qu98Ty75AdL-zC0l-4MFQPidSTCROEoL/view?usp=drive_link", fuzzy=True) #test_images (img)
gdown.download("https://drive.google.com/file/d/1OVJLngs_my3BJNks95-VifrHv-xdIxPW/view?usp=sharing", fuzzy=True) #test_img_msk (img + msk) [created by train dataset]

gdown.download("https://drive.google.com/file/d/1a1ZmzO6ELZt_AfPa5PwuGrC1OEebwyp1/view?usp=sharing", fuzzy=True) #train_annotation_small
gdown.download("https://drive.google.com/file/d/1V_FNruO6Oz35E9FC_ODj1lq3d4ZoAE2Y/view?usp=sharing", fuzzy=True) #val_annotations_small

gdown.download("https://drive.google.com/file/d/1IgkFcKqkTAO91S286xOjylEejvQrzkc4/view?usp=sharing", fuzzy=True) #yolov8.yaml

!unzip /content/train_small.zip
!unzip /content/val_small.zip
# !tar -xvf /content/test_images.tar.gz
!unzip /content/test_img_msk.zip

Downloading...
From: https://drive.google.com/uc?id=1KrbT2KbayuNtsACy7GHmGbUWtikpbSfI
To: /content/train_small.zip
100%|██████████| 210M/210M [00:10<00:00, 19.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MpYdtlbfdCoKW1K1qTcqA_en6RnBtb6g
To: /content/val_small.zip
100%|██████████| 91.8M/91.8M [00:04<00:00, 21.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OVJLngs_my3BJNks95-VifrHv-xdIxPW
To: /content/test_img_msk.zip
100%|██████████| 3.84M/3.84M [00:00<00:00, 108MB/s]
Downloading...
From: https://drive.google.com/uc?id=1a1ZmzO6ELZt_AfPa5PwuGrC1OEebwyp1
To: /content/train_annotation_small.json
100%|██████████| 21.3M/21.3M [00:01<00:00, 19.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1V_FNruO6Oz35E9FC_ODj1lq3d4ZoAE2Y
To: /content/val_annotation_small.json
100%|██████████| 4.61M/4.61M [00:00<00:00, 17.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IgkFcKqkTAO91S286xOjylEejvQrzkc4
To: /content/yolov8.yaml
100%|██████████| 99.0/99.0 [00:

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: content/val_small/masks/000000015133_mask.jpg  
  inflating: content/val_small/masks/000000036218_mask.jpg  
  inflating: content/val_small/masks/000000048332_mask.jpg  
  inflating: content/val_small/masks/000000037706_mask.jpg  
  inflating: content/val_small/masks/000000006237_mask.jpg  
  inflating: content/val_small/masks/000000050357_mask.jpg  
  inflating: content/val_small/masks/000000031921_mask.jpg  
  inflating: content/val_small/masks/000000034652_mask.jpg  
  inflating: content/val_small/masks/000000035483_mask.jpg  
  inflating: content/val_small/masks/000000051993_mask.jpg  
  inflating: content/val_small/masks/000000000108_mask.jpg  
  inflating: content/val_small/masks/000000046902_mask.jpg  
  inflating: content/val_small/masks/000000029294_mask.jpg  
  inflating: content/val_small/masks/000000052964_mask.jpg  
  inflating: content/val_small/masks/000000036788_mask.jpg  
  inflating: cont

# Pathes

In [ ]:
DATA_DIR = '/content'

img_train_dir = os.path.join(DATA_DIR, 'train_small/images/')
mask_train_dir = os.path.join(DATA_DIR, 'train_small/masks/')

img_valid_dir = os.path.join(DATA_DIR, 'val_small/images/')
mask_valid_dir = os.path.join(DATA_DIR, 'val_small/masks/')

img_test_dir = os.path.join(DATA_DIR, 'test_images/')

train_annot = os.path.join(DATA_DIR, 'train_annotation_small.json')
val_annot = os.path.join(DATA_DIR, 'val_annotation_small.json')

# Create datasets

In [136]:
import json
with open(train_annot, 'r') as f:
    data = json.load(f)

In [137]:
df = pd.DataFrame(data['annotations']).drop(['area', 'bbox', 'iscrowd'], axis=1)
df = df.sort_values(by='image_id')
df

,id,image_id,segmentation,category_id
24623,827,95,"[[0.0, 188.0, 17.0, 189.0, 17.0, 211.0, 0.0, 2...",100
11932,820,95,"[[169.0, 272.0, 279.0, 279.0, 278.0, 300.0, 16...",100
53619,823,95,"[[75.0, 215.0, 184.0, 219.0, 184.0, 241.0, 75....",100
44777,828,95,"[[0.0, 261.0, 14.0, 261.0, 14.0, 300.0, 0.0, 3...",100
38013,826,95,"[[0.0, 87.99999999999999, 4.0, 87.999999999999...",100
...,...,...,...,...
21938,2395411,280723,"[[110, 221, 106, 254, 46, 250, 49, 221, 63, 22...",100
60027,2395413,280723,"[[124, 0, 157, 5, 149, 34, 103, 27, 110, 0, 12...",100
33505,2395412,280723,"[[130, 176, 125, 196, 121, 196, 117, 212, 72, ...",100
18817,2395418,280723,"[[89, 24, 102, 26, 101, 36, 87, 35, 89, 24]]",100


In [ ]:
!mkdir /content/yolo_format
!mkdir /content/yolo_format/yolo_txt

mkdir: cannot create directory ‘/content/yolo_format’: File exists


In [139]:
img_ids = df['image_id'].unique()
for id in img_ids:
    yolo_lines = []
    segs = df[df['image_id'] == id]['segmentation']
    cats = df[df['image_id'] == id]['category_id']
    for seg, cat in zip(segs, cats):
        seg[0].insert(0, cat)
        yolo_lines += "".join(str(x) + ' ' for x in seg[0]) + '\n'
    with open(f'/content/yolo_format/yolo_txt/img_{id}.txt', 'w') as f:
        f.writelines(yolo_lines)

In [140]:
!zip -r yolo_txt.zip /content/yolo_format/yolo_txt

Выходные данные были обрезаны до нескольких последних строк (5000).
  adding: content/yolo_format/yolo_txt/img_100053.txt (deflated 53%)
  adding: content/yolo_format/yolo_txt/img_226618.txt (deflated 72%)
  adding: content/yolo_format/yolo_txt/img_236842.txt (deflated 47%)
  adding: content/yolo_format/yolo_txt/img_105099.txt (deflated 80%)
  adding: content/yolo_format/yolo_txt/img_4423.txt (deflated 66%)
  adding: content/yolo_format/yolo_txt/img_130915.txt (deflated 61%)
  adding: content/yolo_format/yolo_txt/img_156210.txt (deflated 69%)
  adding: content/yolo_format/yolo_txt/img_263038.txt (deflated 56%)
  adding: content/yolo_format/yolo_txt/img_113226.txt (deflated 78%)
  adding: content/yolo_format/yolo_txt/img_80989.txt (deflated 69%)
  adding: content/yolo_format/yolo_txt/img_59933.txt (deflated 46%)
  adding: content/yolo_format/yolo_txt/img_213957.txt (deflated 80%)
  adding: content/yolo_format/yolo_txt/img_91797.txt (deflated 75%)
  adding: content/yolo_format/yolo_txt/i